In [2]:
import tools.conf
import psycopg2
from psycopg2 import sql
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('./data/baza.csv', delimiter=';')
df.sort_values(by='data', axis=0, inplace=True)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)


In [143]:
df.head()

,data,dzien,dzien_tyg,nr_tyg,rok,miesiac,zasięg medium,brand,produkt(4),kod_reklamy,opis_reklamy,wydawca_nadawca,typ_reklamy,submedium,dl_mod,gg,mm,koszt,l_emisji,daypart,dł_ujednolicona,godzina_bloku_reklamowego
0,2023-08-01,1,Wtorek,31,2023,8,regionalne,MEDIA MASTER,GRUPA,22194483,PATRZ BARBARA NO ALE..MEGA OKAZJE..SF SAMSUNG ...,SOME GROUP,reklama,SOME MINN Bydgoszcz,29,8,20,310,1,do 9,30,8:00-8:29
1,2023-08-01,1,Wtorek,31,2023,8,miejskie,MEDIA MASTER,GRUPA,22179830,MARIAN A GDZIE MOGĘ KUPIĆ..DOKUPIĆ..ŻEBY TO WS...,PIGLET S.A.,reklama,FROGGY WEATHER Warszawa,14,10,59,311,1,od 9 do 16,15,10:30-10:59
2,2023-08-01,1,Wtorek,31,2023,8,regionalne,MEDIA MASTER,GRUPA,22194487,PATRZ BARBARA NO ALE..MEGA OKAZJE..TV SMART PH...,PIGLET S.A.,reklama,YELLOW HITS Zielona Góra,30,15,22,73,1,od 9 do 16,30,15:00-15:29
3,2023-08-01,1,Wtorek,31,2023,8,regionalne,MEDIA MASTER,GRUPA,22194445,PATRZ BARBARA NO ALE..MEGA OKAZJE..AUT. EKSPRE...,PIGLET S.A.,reklama,YELLOW HITS Zielona Góra,29,14,42,67,1,od 9 do 16,30,14:30-14:59
4,2023-08-01,1,Wtorek,31,2023,8,regionalne,MEDIA MASTER,GRUPA,22194475,BARBARA W - SĄ MEGA OKAZJE..MULTIRABATY..IM WI...,PIGLET S.A.,reklama,YELLOW HITS Zielona Góra,30,13,58,67,1,od 9 do 16,30,13:30-13:59


In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366681 entries, 0 to 366680
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   data                       366681 non-null  object
 1   dzien                      366681 non-null  int64 
 2   dzien_tyg                  366681 non-null  object
 3   nr_tyg                     366681 non-null  int64 
 4   rok                        366681 non-null  int64 
 5   miesiac                    366681 non-null  int64 
 6   zasięg medium              366681 non-null  object
 7   brand                      366681 non-null  object
 8   produkt(4)                 366681 non-null  object
 9   kod_reklamy                366681 non-null  int64 
 10  opis_reklamy               366681 non-null  object
 11  wydawca_nadawca            366681 non-null  object
 12  typ_reklamy                366681 non-null  object
 13  submedium                  366681 non-null  

In [145]:
dow2 = ['Poniedziałek', 'Wtorek', 'Środa', 'Czwartek', 'Piątek',
        'Sobota', 'Niedziela']

In [146]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()


for day in dow2:
    cur.execute(
        sql.SQL('INSERT INTO {} (dow_name) VALUES (%s)')
        .format(sql.Identifier('pl_dow_names')), (day,)
    )

conn.commit()
conn.close()

In [147]:
months = ['Styczeń', 'Luty', 'Marzec', 'Kwiecień', 'Maj',
          'Czerwiec', 'Lipiec', 'Sierpień', 'Wrzesień',
          'Październik', 'Listopad', 'Grudzień']

In [148]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()


query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for month in months:
    cur.execute(
        query.format(
            table=sql.Identifier('pl_month_names'),
            field=sql.Identifier('month_name')), (month,)
    )

conn.commit()
conn.close()


In [149]:
dates = df['data'].unique()
dates

array(['2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04',
       '2023-08-05', '2023-08-06', '2023-08-07', '2023-08-08',
       '2023-08-09', '2023-08-10', '2023-08-11', '2023-08-12',
       '2023-08-13', '2023-08-14', '2023-08-15', '2023-08-16',
       '2023-08-17', '2023-08-18', '2023-08-19', '2023-08-20',
       '2023-08-21', '2023-08-22', '2023-08-23', '2023-08-24',
       '2023-08-25', '2023-08-26', '2023-08-27', '2023-08-28',
       '2023-08-29', '2023-08-30', '2023-08-31', '2023-09-01',
       '2023-09-02', '2023-09-03', '2023-09-04', '2023-09-05',
       '2023-09-06', '2023-09-07', '2023-09-08', '2023-09-09',
       '2023-09-10', '2023-09-11', '2023-09-12', '2023-09-13',
       '2023-09-14', '2023-09-15', '2023-09-16', '2023-09-17',
       '2023-09-18', '2023-09-19', '2023-09-20', '2023-09-21',
       '2023-09-22', '2023-09-23', '2023-09-24', '2023-09-25',
       '2023-09-26', '2023-09-27', '2023-09-28', '2023-09-29',
       '2023-09-30', '2023-10-01', '2023-10-02', '2023-

In [150]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()


query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for date in dates:
    cur.execute(
        query.format(
            table=sql.Identifier('date_time'),
            field=sql.Identifier('date')), (date,)
    )

conn.commit()
conn.close()

In [8]:
brands = df['brand'].sort_values().unique()
brands

array(['EURO APPLIANCES', 'MEDIA MASTER', 'MEDIA SHOP', 'NEWNET'],
      dtype=object)

In [10]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query = sql.SQL('INSERT INTO {table} ({field}) VALUES(%s)')

for brand in brands:
    cur.execute(
        query.format(
            table=sql.Identifier('brands'),
            field=sql.Identifier('brand')), (brand,)
    )

conn.commit()
conn.close()

In [13]:
lengths = df['dł_ujednolicona'].sort_values().unique()
lengths

array([15, 20, 30])

In [15]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for length in lengths:
    cur.execute(
        query.format(
            table=sql.Identifier('unified_lenghts'),
            field=sql.Identifier('length')), (str(length),)
    )
    
conn.commit()
conn.close()

In [17]:
dayparts = df['daypart'].unique()
dayparts

array(['do 9', 'od 9 do 16', 'po 16'], dtype=object)

In [19]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for daypart in dayparts:
    cur.execute(
        query.format(
            table=sql.Identifier('dayparts'),
            field=sql.Identifier('daypart')), (str(daypart),)
    )
    
conn.commit()
conn.close()

In [22]:
product_types = df['produkt(4)'].sort_values().unique()
product_types

array(['AGD, RTV, ELEKTRONIKA, FOTOGRAFIA, KOMPUTERY', 'GRUPA',
       'OGŁOSZENIA O PRACY'], dtype=object)

In [ ]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for product in product_types:
    cur.execute(
        query.format(
            table=sql.Identifier('product_types'),
            field=sql.Identifier('product_type')), (str(product),)
    )
    
conn.commit()
conn.close()